# EPL DATA ANALYSIS

## Introduction 

The purpose of this notebook is to analyze the impact of covid-19 to the performance of English Premier League (EPL) teams. The dataset contained match information from : 
* 2018/2019 season (1 season before covid-19 interruption)
* 2019/2020 season (season where covid interrupted the league)
* 2020/2021 season (1 season after covid-19 interruption)

Information regarding the date of covid-19 interruption : 
* last match before covid-19 interruption = Leicester vs Aston Villa 10 March 2020 (match ID 46889)
* first match before covid-19 interruption = Aston Villa vs Sheffield United 18 June 2020 (match ID 46875)

Analysis objectives : 
* How the absence of spectators had impacted performance of EPL teams 
    * Home/Away Possession
    * Home/Away Shots on Target
    * Home/Away Shots
    * Home/Away Goals Scored per Game
    * Home/Away Win Rate
    * Home/Away Shot on Target % = Shots on Target / Shots
    * Home/Away Quantity Conversion Rate = Goals / Shots 
    * Home/Away Quality Conversion Rate  = Goals / Shots on Target
    * Home/Away Win Rate = No. of win / No. of games (Home/Away)
    * Home/Away Points per Game (PpG) = Average points/game (Home / Away)
* Relation of Ball Possession % vs Goal Conversion Rate
* Compare the performance metrics between "big-6" and "non big-6" teams before and after covid-19. (note : Big-6 = "Manchester United", "Manchester City", "Liverpool", "Arsenal", "Chelsea", "Tottenham Hotspurs")

In [12]:
import pandas as pd

#Set float number just have two decimals
pd.set_option('display.float_format','{:.2f}' .format)

In [13]:
epl_data = pd.read_csv('epl_data_cleaned.csv', index_col=[0])
epl_data

,match_id,match_date,matchweek,home_team,away_team,season,home_score,away_score,home_possession,away_possession,home_shots_on_target,away_shots_on_target,home_shots,away_shots,home_points,away_points
0,38309,2018-08-11,Matchweek 1,AFC Bournemouth,Cardiff City,2018/2019,2,0,62.90,37.10,4,1,12,10,3,0
1,38310,2018-08-11,Matchweek 1,Fulham,Crystal Palace,2018/2019,0,2,66.30,33.70,6,10,15,12,0,3
2,38311,2018-08-11,Matchweek 1,Huddersfield Town,Chelsea,2018/2019,0,3,37.20,62.80,1,4,6,13,0,3
3,38313,2018-08-11,Matchweek 1,Manchester United,Leicester City,2018/2019,2,1,46.30,53.70,6,4,8,13,3,0
4,38314,2018-08-11,Matchweek 1,Newcastle United,Tottenham Hotspur,2018/2019,1,2,40.40,59.60,2,5,15,15,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135,59272,2021-05-23,Matchweek 38,Manchester City,Everton,2020/2021,5,0,67.70,32.30,11,3,21,8,3,0
1136,59273,2021-05-23,Matchweek 38,Sheffield United,Burnley,2020/2021,1,0,43.00,57.00,3,3,12,10,3,0
1137,59274,2021-05-23,Matchweek 38,West Ham United,Southampton,2020/2021,3,0,36.90,63.10,7,5,14,17,3,0
1138,59275,2021-05-23,Matchweek 38,Wolverhampton Wanderers,Manchester United,2020/2021,1,2,57.20,42.80,4,4,14,9,0,3


## Calculate Key Stats 

Some of the key stats required for further analysis are : 
* Shot on Target % = Shots on Target / Shots
* Quantity Conversion Rate = Goals / Shots 
* Quality Conversion Rate  = Goals / Shots on Target

In [14]:
def calculate_key_stats(df, num, div,stats_name,percentage=True):
    if percentage == True:
        #home key stats
        df['home_'+stats_name] = (df['home_'+num]/df['home_'+div]) * 100
        #away key stats
        df['away_'+stats_name] = (df['away_'+num]/df['away_'+div]) * 100
    else:
        #home key stats
        df['home_'+stats_name] = df['home_'+num]/df['home_'+div]
        #away key stats
        df['away_'+stats_name] = df['away_'+num]/df['away_'+div]
    
    #handling 0/0 value (if any)
    df = df.fillna(0, axis=1)
    
    return df

In [15]:
#calculate shot on target %
calculate_key_stats(df=epl_data, num='shots_on_target',div='shots',stats_name='SoT%')

#calculate quantity conversion rate
calculate_key_stats(df=epl_data, num='score',div='shots',stats_name='quan_CR', percentage=False)

#calculate quality conversion rate
calculate_key_stats(df=epl_data, num='score',div='shots_on_target',stats_name='qual_CR', percentage=False)

,match_id,match_date,matchweek,home_team,away_team,season,home_score,away_score,home_possession,away_possession,...,home_shots,away_shots,home_points,away_points,home_SoT%,away_SoT%,home_quan_CR,away_quan_CR,home_qual_CR,away_qual_CR
0,38309,2018-08-11,Matchweek 1,AFC Bournemouth,Cardiff City,2018/2019,2,0,62.90,37.10,...,12,10,3,0,33.33,10.00,0.17,0.00,0.50,0.00
1,38310,2018-08-11,Matchweek 1,Fulham,Crystal Palace,2018/2019,0,2,66.30,33.70,...,15,12,0,3,40.00,83.33,0.00,0.17,0.00,0.20
2,38311,2018-08-11,Matchweek 1,Huddersfield Town,Chelsea,2018/2019,0,3,37.20,62.80,...,6,13,0,3,16.67,30.77,0.00,0.23,0.00,0.75
3,38313,2018-08-11,Matchweek 1,Manchester United,Leicester City,2018/2019,2,1,46.30,53.70,...,8,13,3,0,75.00,30.77,0.25,0.08,0.33,0.25
4,38314,2018-08-11,Matchweek 1,Newcastle United,Tottenham Hotspur,2018/2019,1,2,40.40,59.60,...,15,15,0,3,13.33,33.33,0.07,0.13,0.50,0.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135,59272,2021-05-23,Matchweek 38,Manchester City,Everton,2020/2021,5,0,67.70,32.30,...,21,8,3,0,52.38,37.50,0.24,0.00,0.45,0.00
1136,59273,2021-05-23,Matchweek 38,Sheffield United,Burnley,2020/2021,1,0,43.00,57.00,...,12,10,3,0,25.00,30.00,0.08,0.00,0.33,0.00
1137,59274,2021-05-23,Matchweek 38,West Ham United,Southampton,2020/2021,3,0,36.90,63.10,...,14,17,3,0,50.00,29.41,0.21,0.00,0.43,0.00
1138,59275,2021-05-23,Matchweek 38,Wolverhampton Wanderers,Manchester United,2020/2021,1,2,57.20,42.80,...,14,9,0,3,28.57,44.44,0.07,0.22,0.25,0.50


## Calculate Points

In [16]:
def get_points(df):
    #home win
    df.loc[df['home_score']>df['away_score'], 'home_points'] = 3
    df.loc[df['home_score']>df['away_score'], 'away_points'] = 0

    #away win
    df.loc[df['home_score']<df['away_score'], 'home_points'] = 0
    df.loc[df['home_score']<df['away_score'], 'away_points'] = 3
    
    #draw
    df.loc[df['home_score']==df['away_score'], 'home_points'] = 1
    df.loc[df['home_score']==df['away_score'], 'away_points'] = 1
    
    return df
        
#calculate points for each game
get_points(epl_data)

,match_id,match_date,matchweek,home_team,away_team,season,home_score,away_score,home_possession,away_possession,...,home_shots,away_shots,home_points,away_points,home_SoT%,away_SoT%,home_quan_CR,away_quan_CR,home_qual_CR,away_qual_CR
0,38309,2018-08-11,Matchweek 1,AFC Bournemouth,Cardiff City,2018/2019,2,0,62.90,37.10,...,12,10,3,0,33.33,10.00,0.17,0.00,0.50,0.00
1,38310,2018-08-11,Matchweek 1,Fulham,Crystal Palace,2018/2019,0,2,66.30,33.70,...,15,12,0,3,40.00,83.33,0.00,0.17,0.00,0.20
2,38311,2018-08-11,Matchweek 1,Huddersfield Town,Chelsea,2018/2019,0,3,37.20,62.80,...,6,13,0,3,16.67,30.77,0.00,0.23,0.00,0.75
3,38313,2018-08-11,Matchweek 1,Manchester United,Leicester City,2018/2019,2,1,46.30,53.70,...,8,13,3,0,75.00,30.77,0.25,0.08,0.33,0.25
4,38314,2018-08-11,Matchweek 1,Newcastle United,Tottenham Hotspur,2018/2019,1,2,40.40,59.60,...,15,15,0,3,13.33,33.33,0.07,0.13,0.50,0.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135,59272,2021-05-23,Matchweek 38,Manchester City,Everton,2020/2021,5,0,67.70,32.30,...,21,8,3,0,52.38,37.50,0.24,0.00,0.45,0.00
1136,59273,2021-05-23,Matchweek 38,Sheffield United,Burnley,2020/2021,1,0,43.00,57.00,...,12,10,3,0,25.00,30.00,0.08,0.00,0.33,0.00
1137,59274,2021-05-23,Matchweek 38,West Ham United,Southampton,2020/2021,3,0,36.90,63.10,...,14,17,3,0,50.00,29.41,0.21,0.00,0.43,0.00
1138,59275,2021-05-23,Matchweek 38,Wolverhampton Wanderers,Manchester United,2020/2021,1,2,57.20,42.80,...,14,9,0,3,28.57,44.44,0.07,0.22,0.25,0.50


## Calculate Win Rate

In [17]:
def get_win_rate(df):
    #home win
    df.loc[df['home_score']>df['away_score'], 'home_win_rate'] = 1
    df.loc[df['home_score']>df['away_score'], 'away_win_rate'] = 0

    #away win
    df.loc[df['home_score']<df['away_score'], 'home_win_rate'] = 0
    df.loc[df['home_score']<df['away_score'], 'away_win_rate'] = 1
    
    #draw
    df.loc[df['home_score']==df['away_score'], 'home_win_rate'] = 0
    df.loc[df['home_score']==df['away_score'], 'away_win_rate'] = 0
    
    return df

#calculate win rate
get_win_rate(epl_data)

,match_id,match_date,matchweek,home_team,away_team,season,home_score,away_score,home_possession,away_possession,...,home_points,away_points,home_SoT%,away_SoT%,home_quan_CR,away_quan_CR,home_qual_CR,away_qual_CR,home_win_rate,away_win_rate
0,38309,2018-08-11,Matchweek 1,AFC Bournemouth,Cardiff City,2018/2019,2,0,62.90,37.10,...,3,0,33.33,10.00,0.17,0.00,0.50,0.00,1.00,0.00
1,38310,2018-08-11,Matchweek 1,Fulham,Crystal Palace,2018/2019,0,2,66.30,33.70,...,0,3,40.00,83.33,0.00,0.17,0.00,0.20,0.00,1.00
2,38311,2018-08-11,Matchweek 1,Huddersfield Town,Chelsea,2018/2019,0,3,37.20,62.80,...,0,3,16.67,30.77,0.00,0.23,0.00,0.75,0.00,1.00
3,38313,2018-08-11,Matchweek 1,Manchester United,Leicester City,2018/2019,2,1,46.30,53.70,...,3,0,75.00,30.77,0.25,0.08,0.33,0.25,1.00,0.00
4,38314,2018-08-11,Matchweek 1,Newcastle United,Tottenham Hotspur,2018/2019,1,2,40.40,59.60,...,0,3,13.33,33.33,0.07,0.13,0.50,0.40,0.00,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135,59272,2021-05-23,Matchweek 38,Manchester City,Everton,2020/2021,5,0,67.70,32.30,...,3,0,52.38,37.50,0.24,0.00,0.45,0.00,1.00,0.00
1136,59273,2021-05-23,Matchweek 38,Sheffield United,Burnley,2020/2021,1,0,43.00,57.00,...,3,0,25.00,30.00,0.08,0.00,0.33,0.00,1.00,0.00
1137,59274,2021-05-23,Matchweek 38,West Ham United,Southampton,2020/2021,3,0,36.90,63.10,...,3,0,50.00,29.41,0.21,0.00,0.43,0.00,1.00,0.00
1138,59275,2021-05-23,Matchweek 38,Wolverhampton Wanderers,Manchester United,2020/2021,1,2,57.20,42.80,...,0,3,28.57,44.44,0.07,0.22,0.25,0.50,0.00,1.00


## Change Datatype

In [18]:
epl_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1140 entries, 0 to 1139
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   match_id              1140 non-null   int64  
 1   match_date            1140 non-null   object 
 2   matchweek             1140 non-null   object 
 3   home_team             1140 non-null   object 
 4   away_team             1140 non-null   object 
 5   season                1140 non-null   object 
 6   home_score            1140 non-null   int64  
 7   away_score            1140 non-null   int64  
 8   home_possession       1140 non-null   float64
 9   away_possession       1140 non-null   float64
 10  home_shots_on_target  1140 non-null   int64  
 11  away_shots_on_target  1140 non-null   int64  
 12  home_shots            1140 non-null   int64  
 13  away_shots            1140 non-null   int64  
 14  home_points           1140 non-null   int64  
 15  away_points          

In [19]:
string_cols = ['matchweek','home_team','away_team','season']
int_cols = ['home_score','away_score','home_shots_on_target','away_shots_on_target',
            'home_shots','away_shots','home_points','away_points']
float_cols = ['home_possession','away_possession','home_SoT%','away_SoT%','home_quan_CR','away_quan_CR' \
              ,'home_qual_CR','away_qual_CR','home_win_rate','away_win_rate']
datetime_cols = 'match_date'

#convert to corresponding datatype
epl_data[string_cols] = epl_data[string_cols].astype("string")
epl_data[int_cols] = epl_data[int_cols].astype("int")
epl_data[float_cols] = epl_data[float_cols].astype("float")

#convert object to datetime object
epl_data['match_date'] = pd.to_datetime(epl_data['match_date'], format='%Y-%m-%d')

epl_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1140 entries, 0 to 1139
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   match_id              1140 non-null   int64         
 1   match_date            1140 non-null   datetime64[ns]
 2   matchweek             1140 non-null   string        
 3   home_team             1140 non-null   string        
 4   away_team             1140 non-null   string        
 5   season                1140 non-null   string        
 6   home_score            1140 non-null   int32         
 7   away_score            1140 non-null   int32         
 8   home_possession       1140 non-null   float64       
 9   away_possession       1140 non-null   float64       
 10  home_shots_on_target  1140 non-null   int32         
 11  away_shots_on_target  1140 non-null   int32         
 12  home_shots            1140 non-null   int32         
 13  away_shots        

In [20]:
#label match before and after covid19
epl_data.loc[epl_data['match_date']<='2020-03-10','covid19'] = 'before'
epl_data.loc[epl_data['match_date']>'2020-03-10','covid19'] = 'after'

epl_data

,match_id,match_date,matchweek,home_team,away_team,season,home_score,away_score,home_possession,away_possession,...,away_points,home_SoT%,away_SoT%,home_quan_CR,away_quan_CR,home_qual_CR,away_qual_CR,home_win_rate,away_win_rate,covid19
0,38309,2018-08-11,Matchweek 1,AFC Bournemouth,Cardiff City,2018/2019,2,0,62.90,37.10,...,0,33.33,10.00,0.17,0.00,0.50,0.00,1.00,0.00,before
1,38310,2018-08-11,Matchweek 1,Fulham,Crystal Palace,2018/2019,0,2,66.30,33.70,...,3,40.00,83.33,0.00,0.17,0.00,0.20,0.00,1.00,before
2,38311,2018-08-11,Matchweek 1,Huddersfield Town,Chelsea,2018/2019,0,3,37.20,62.80,...,3,16.67,30.77,0.00,0.23,0.00,0.75,0.00,1.00,before
3,38313,2018-08-11,Matchweek 1,Manchester United,Leicester City,2018/2019,2,1,46.30,53.70,...,0,75.00,30.77,0.25,0.08,0.33,0.25,1.00,0.00,before
4,38314,2018-08-11,Matchweek 1,Newcastle United,Tottenham Hotspur,2018/2019,1,2,40.40,59.60,...,3,13.33,33.33,0.07,0.13,0.50,0.40,0.00,1.00,before
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135,59272,2021-05-23,Matchweek 38,Manchester City,Everton,2020/2021,5,0,67.70,32.30,...,0,52.38,37.50,0.24,0.00,0.45,0.00,1.00,0.00,after
1136,59273,2021-05-23,Matchweek 38,Sheffield United,Burnley,2020/2021,1,0,43.00,57.00,...,0,25.00,30.00,0.08,0.00,0.33,0.00,1.00,0.00,after
1137,59274,2021-05-23,Matchweek 38,West Ham United,Southampton,2020/2021,3,0,36.90,63.10,...,0,50.00,29.41,0.21,0.00,0.43,0.00,1.00,0.00,after
1138,59275,2021-05-23,Matchweek 38,Wolverhampton Wanderers,Manchester United,2020/2021,1,2,57.20,42.80,...,3,28.57,44.44,0.07,0.22,0.25,0.50,0.00,1.00,after


## Data Analysis

In [42]:
def evaluate_stats(df,negative_changes=True):
    #evaluate changes 
    ev = df.copy()
    
    if negative_changes == True:
    
        ev.loc[ev['changes']>0,'effect'] = 'positive'  
        ev.loc[ev['changes']<0,'effect'] = 'negative'   
        ev.loc[ev['changes']==0,'effect'] = 'not affected'  
    
    else:
        
        ev.loc[ev['changes']<0,'effect'] = 'positive'  
        ev.loc[ev['changes']>0,'effect'] = 'negative'   
        ev.loc[ev['changes']==0,'effect'] = 'not affected'  
    
    evaluation = ev.value_counts('effect').to_frame(name='count')
    return evaluation

In [43]:
def get_home_stats(stats_name,df=epl_data,team=filtered_teams,evaluate=True):
    
    print('The effect of covid-19 on EPL home teams ' + stats_name)
    
    home_stats = pd.DataFrame()
    
    #home stats before covid-19
    home_stats['home_'+stats_name+'_before'] = \
        df.loc[df['covid19']=='before'].groupby('home_team')['home_'+stats_name].mean()
    
    #home stats after covid-19
    home_stats['home_'+stats_name+'_after'] = \
        df.loc[df['covid19']=='after'].groupby('home_team')['home_'+stats_name].mean()
    
    #filter team 
    home_stats = home_stats.loc[team]
    
    #changes in home stats
    home_stats['changes'] = home_stats['home_'+stats_name+'_after'] - home_stats['home_'+stats_name+'_before']
    
    #sort by biggest change ascending
    home_stats = home_stats.sort_values('changes', ascending=True)
    
    if evaluate == True:
        display(evaluate_stats(home_stats))
    
    return home_stats

In [44]:
def get_away_stats(stats_name,df=epl_data,team=filtered_teams,evaluate=True):
    
    print('The effect of covid-19 on EPL away teams ' + stats_name)
    
    away_stats = pd.DataFrame()
    
    #away stats before covid-19
    away_stats['away_'+stats_name+'_before'] = \
        df.loc[df['covid19']=='before'].groupby('away_team')['away_'+stats_name].mean()
    
    #away stats after covid-19
    away_stats['away_'+stats_name+'_after'] = \
        df.loc[df['covid19']=='after'].groupby('away_team')['away_'+stats_name].mean()
    
    #filter team 
    away_stats = away_stats.loc[team]
    
    #changes in home stats
    away_stats['changes'] = away_stats['away_'+stats_name+'_after'] - away_stats['away_'+stats_name+'_before']
    
    #sort by biggest change ascending
    away_stats = away_stats.sort_values('changes', ascending=True)
    
    if evaluate == True:
        display(evaluate_stats(away_stats))
    
    return away_stats

### Filter EPL Teams

The analysis will only observe the teams which had participated in all 3 seasons.

In [21]:
#get teams name for every season since 2018/2019 
teams_18_19 = set(epl_data.loc[epl_data['season']=='2018/2019']['home_team'].unique())
teams_19_20 = set(epl_data.loc[epl_data['season']=='2019/2020']['home_team'].unique())
teams_20_21 = set(epl_data.loc[epl_data['season']=='2020/2021']['home_team'].unique())

#filter the team that had participated in all 3 seasons
filtered_teams = list(teams_18_19.intersection(teams_19_20,teams_20_21))
filtered_teams

['Chelsea',
 'Tottenham Hotspur',
 'Leicester City',
 'Newcastle United',
 'West Ham United',
 'Wolverhampton Wanderers',
 'Liverpool',
 'Everton',
 'Manchester United',
 'Manchester City',
 'Southampton',
 'Arsenal',
 'Brighton and Hove Albion',
 'Burnley',
 'Crystal Palace']

### Home / Away Ball Possession %

In [30]:
get_home_stats('possession')

The effect of covid-19 on EPL home teams possession


,count
effect,
negative,8
positive,7


,home_possession_before,home_possession_after,changes
home_team,,,
Tottenham Hotspur,58.73,50.82,-7.91
Arsenal,59.00,52.74,-6.26
Everton,52.55,47.93,-4.63
Crystal Palace,45.98,41.80,-4.18
West Ham United,47.63,43.60,-4.03
Chelsea,62.71,60.49,-2.21
Manchester City,67.76,66.72,-1.04
Leicester City,55.70,55.65,-0.04
Newcastle United,41.60,41.93,0.33


In [31]:
get_away_stats('possession')

The effect of covid-19 on EPL away teams possession


,count
effect,
positive,8
negative,7


,away_possession_before,away_possession_after,changes
away_team,,,
Everton,49.26,45.14,-4.12
Manchester City,66.74,62.74,-4.00
West Ham United,45.55,42.45,-3.10
Crystal Palace,43.43,40.45,-2.98
Arsenal,55.41,52.71,-2.69
Tottenham Hotspur,54.25,51.94,-2.31
Liverpool,62.07,61.16,-0.91
Brighton and Hove Albion,46.15,46.90,0.75
Newcastle United,35.99,36.77,0.78


### Home/Away Shots on Target

In [32]:
get_home_stats('shots_on_target')

The effect of covid-19 on EPL home teams shots_on_target


,count
effect,
negative,10
positive,5


,home_shots_on_target_before,home_shots_on_target_after,changes
home_team,,,
Everton,4.94,3.79,-1.15
West Ham United,4.88,4.04,-0.84
Liverpool,6.71,5.91,-0.79
Manchester United,6.47,5.70,-0.77
Burnley,4.00,3.26,-0.74
Manchester City,7.38,6.68,-0.70
Arsenal,4.88,4.30,-0.58
Crystal Palace,3.82,3.26,-0.56
Leicester City,5.06,4.78,-0.28


In [33]:
get_away_stats('shots_on_target')

The effect of covid-19 on EPL away teams shots_on_target


,count
effect,
positive,9
negative,6


,away_shots_on_target_before,away_shots_on_target_after,changes
away_team,,,
Tottenham Hotspur,4.59,3.57,-1.02
Manchester City,6.35,5.43,-0.92
Wolverhampton Wanderers,3.88,3.33,-0.55
Everton,4.15,3.65,-0.49
Leicester City,5.03,4.67,-0.36
Brighton and Hove Albion,3.50,3.17,-0.33
Crystal Palace,3.39,3.46,0.06
Southampton,4.36,4.46,0.09
Newcastle United,3.09,3.30,0.22


### Home/Away Shots

In [34]:
get_home_stats('shots')

The effect of covid-19 on EPL home teams shots


,count
effect,
negative,13
positive,2


,home_shots_before,home_shots_after,changes
home_team,,,
Crystal Palace,13.79,9.48,-4.32
Everton,14.61,11.08,-3.52
Manchester City,20.81,17.68,-3.13
Tottenham Hotspur,15.00,12.17,-2.83
Chelsea,17.29,15.30,-1.99
Leicester City,14.94,13.00,-1.94
Burnley,11.50,10.30,-1.20
Manchester United,15.62,14.43,-1.18
Newcastle United,13.36,12.21,-1.16


In [50]:
get_away_stats('shots')

The effect of covid-19 on EPL away teams shots


,count
effect,
negative,9
positive,6


,away_shots_before,away_shots_after,changes
away_team,,,
Everton,11.85,9.39,-2.46
Wolverhampton Wanderers,11.79,9.88,-1.91
Southampton,12.45,11.21,-1.25
Tottenham Hotspur,11.59,10.57,-1.02
Manchester City,16.41,15.70,-0.72
Chelsea,15.15,14.50,-0.65
Crystal Palace,9.42,9.17,-0.26
Newcastle United,8.97,8.83,-0.14
Brighton and Hove Albion,10.26,10.22,-0.05


In [53]:
get_home_stats('shots_on_target',team=['Liverpool'],evaluate=False)

The effect of covid-19 on EPL home teams shots_on_target


,home_shots_on_target_before,home_shots_on_target_after,changes
home_team,,,
Liverpool,6.71,5.91,-0.79
